<a href="https://colab.research.google.com/github/pritip22/DSA_LAB/blob/main/AIDS2_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Uploading of Data Set anf checking for Null Values


In [3]:
import pandas as pd

# Load the dataset
data = pd.read_csv('/content/dairy_dataset.csv')


In [4]:
# Check for missing data
print(data.isnull().sum())

Location                               0
Total Land Area (acres)                0
Number of Cows                         0
Farm Size                              0
Date                                   0
Product ID                             0
Product Name                           0
Brand                                  0
Quantity (liters/kg)                   0
Price per Unit                         0
Total Value                            0
Shelf Life (days)                      0
Storage Condition                      0
Production Date                        0
Expiration Date                        0
Quantity Sold (liters/kg)              0
Price per Unit (sold)                  0
Approx. Total Revenue(INR)             0
Customer Location                      0
Sales Channel                          0
Quantity in Stock (liters/kg)          0
Minimum Stock Threshold (liters/kg)    0
Reorder Quantity (liters/kg)           0
dtype: int64


In [5]:
data.fillna(method='ffill', inplace=True)

<ipython-input-5-519281724d28>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


Feature Selection

In [6]:
features = [
    'Location', 'Total Land Area (acres)', 'Number of Cows', 'Farm Size',
    'Date', 'Product Name', 'Brand', 'Quantity (liters/kg)', 'Price per Unit',
    'Production Date', 'Expiration Date', 'Sales Channel',
    'Quantity in Stock (liters/kg)', 'Minimum Stock Threshold (liters/kg)'
]

target = 'Quantity Sold (liters/kg)'


Feature Engineering: Transformation of Data and Encoding Categorial Variables


In [7]:
from sklearn.preprocessing import LabelEncoder

# Convert dates to numerical values
data['Date'] = pd.to_datetime(data['Date']).astype(int) / 10**9  # Convert date to seconds
data['Production Date'] = pd.to_datetime(data['Production Date']).astype(int) / 10**9
data['Expiration Date'] = pd.to_datetime(data['Expiration Date']).astype(int) / 10**9

# Encode categorical variables
label_encoder = LabelEncoder()
data['Location'] = label_encoder.fit_transform(data['Location'])
data['Product Name'] = label_encoder.fit_transform(data['Product Name'])
data['Brand'] = label_encoder.fit_transform(data['Brand'])
data['Sales Channel'] = label_encoder.fit_transform(data['Sales Channel'])
data['Farm Size'] = label_encoder.fit_transform(data['Farm Size'])


Data Normalization

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[features])


Splitting Data for Training and Testing

In [9]:
from sklearn.model_selection import train_test_split

X = scaled_data  # Feature set
y = data[target].values  # Target (Quantity Sold)

# Split into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Building the Neural Network Model (Using FeedForward Neural Network)

In [24]:
from keras.models import Sequential
from keras.layers import Dense

# Build the model
model = Sequential()

# Add input layer (number of input features)
model.add(Dense(units=128, activation='relu', input_shape=(X_train.shape[1],)))

# Add hidden layers
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=32, activation='relu'))

# Add output layer (since we are predicting a continuous value)
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)




/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 104781.4531 - val_loss: 50682.3867
Epoch 2/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 44994.7383 - val_loss: 45940.0898
Epoch 3/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 45217.3984 - val_loss: 40798.4688
Epoch 4/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 36661.8242 - val_loss: 31988.9688
Epoch 5/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 27686.7480 - val_loss: 14908.0898
Epoch 6/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9904.9004 - val_loss: 1558.2637
Epoch 7/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 861.2340 - val_loss: 125.6844
Epoch 8/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 97.9389 - val_loss: 44.5292
Epoch 9/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 33.5822 - val_loss: 20.0253
Epoch 10/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 16.2435 - val_loss: 13.5730
Epoch 11/50
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 12.4005 - val_loss: 10.8120
Epoch 12/50
87/

Model Evaluation

In [25]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'Root Mean Squared Error: {rmse}')

# Calculate R-squared (Testing Accuracy)
r2 = r2_score(y_test, y_pred)
print(f'R-squared (Testing Accuracy): {r2}')


28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Mean Absolute Error: 0.5337569003763226
Root Mean Squared Error: 0.8144472683318962
R-squared (Testing Accuracy): 0.99998539686203


Model Deployment and Building a predictive system


In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Assuming 'model' is already trained

# Re-use the dataset or the list of unique values seen during training
# Here's an example of how to extend the label encoding with unseen labels
possible_locations = ['Telangana', 'Mumbai', 'Karnataka', 'Andhra Pradesh']  # Add any other locations
possible_products = ['Ice Cream', 'Milk', 'Yogurt']  # Example of products, extend as necessary
possible_brands = ['Dodla Dairy', 'Amul', 'Nandini']  # Add brands seen during training
possible_sales_channels = ['Wholesale', 'Retail']  # Sales channels seen during training
possible_farm_sizes = ['Small', 'Medium', 'Large']

# Re-train label encoders with all possible categories
label_encoder_location = LabelEncoder()
label_encoder_location.fit(possible_locations)

label_encoder_product = LabelEncoder()
label_encoder_product.fit(possible_products)

label_encoder_brand = LabelEncoder()
label_encoder_brand.fit(possible_brands)

label_encoder_sales_channel = LabelEncoder()
label_encoder_sales_channel.fit(possible_sales_channels)

label_encoder_farm_size = LabelEncoder()
label_encoder_farm_size.fit(possible_farm_sizes)

# Prediction function
def make_prediction(input_data):
    # Convert input data to a DataFrame
    input_df = pd.DataFrame([input_data])

    # Convert date columns to timestamps
    input_df['Date'] = pd.to_datetime(input_df['Date']).astype(int) / 10**9
    input_df['Production Date'] = pd.to_datetime(input_df['Production Date']).astype(int) / 10**9
    input_df['Expiration Date'] = pd.to_datetime(input_df['Expiration Date']).astype(int) / 10**9

    # Label encoding (make sure unseen labels are handled properly)
    input_df['Location'] = label_encoder_location.transform(input_df['Location'])
    input_df['Product Name'] = label_encoder_product.transform(input_df['Product Name'])
    input_df['Brand'] = label_encoder_brand.transform(input_df['Brand'])
    input_df['Sales Channel'] = label_encoder_sales_channel.transform(input_df['Sales Channel'])
    input_df['Farm Size'] = label_encoder_farm_size.transform(input_df['Farm Size'])

    # Scale the input data
    input_scaled = scaler.transform(input_df[features])

    # Make the prediction using the recently trained model
    prediction = model.predict(input_scaled)
    return prediction[0][0]

# Sample input for prediction (you can modify this as needed)
sample_input = {
    "Location": "Mumbai",
    "Total Land Area (acres)": 310.84,
    "Number of Cows": 96,
    "Farm Size": "Medium",
    "Date": "2022-02-17",
    "Product Name": "Ice Cream",
    "Brand": "Dodla Dairy",
    "Quantity (liters/kg)": 222.40,
    "Price per Unit": 85.72,
    "Production Date": "2021-12-27",
    "Expiration Date": "2022-01-21",
    "Sales Channel": "Wholesale",
    "Quantity in Stock (liters/kg)": 215,
    "Minimum Stock Threshold (liters/kg)": 19.55
}

# Predict the output using the model that was just trained
predicted_quantity_sold = make_prediction(sample_input)
print(f'Predicted Quantity Sold: {predicted_quantity_sold:.2f} liters/kg')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted Quantity Sold: 6.90 liters/kg
